# Convert Natural Language to SQL Query with ReAct Agent

This notebook will show 
- how to use ReAct agent to convert natural language to SQL query
- how to use a customized NL2SQL algorithm, DAIL-SQL, in AgentScope 

## Background

NL2SQl is a challenging task in both database and natural language processing communities. It aims to convert natural language question into SQL query. In research community, there are many works exploring the potential of LLMs in NL2SQL, and it would be very interesting to explore this task with LLM-empowered agents. 

In this notebook, we try to equip the ReAct agent with a SOTA NL2SQL algorithm, DAIL-SQL, to see if the agent-level reasoning could bring any insights to this task. The implementation of DAIL-SQL refer to its [GitHub repository](https://github.com/BeachWang/DAIL-SQL) and [paper](https://arxiv.org/abs/2308.15363).

## Prerequisites

- Follow [READMD.md](https://github.com/modelscope/agentscope) to install AgentScope.
- Install the third-party libraries used in [DAIL-SQL](https://github.com/BeachWang/DAIL-SQL/blob/main/requirements.txt).
- Prepare a model configuration. AgentScope supports both local deployed model services (CPU or GPU) and third-party services. More details and example model configurations please refer to our [tutorial](https://modelscope.github.io/agentscope/en/tutorial/203-model.html).
- Get familiar with service functions, ServiceToolkit module and build-in ReAct agent. The corresponding example can be found in [ReAct agent example](../conversation_with_react_agent/main.ipynb).

## Note

- The example is tested with the following models. For other models, you may need to adjust the prompt.
    - gpt-4 (The model that [DAIL-SQL](https://github.com/BeachWang/DAIL-SQL) is built on)
    - gpt-3.5-turbo   

## Step 1: Initializing model config and SQL database
First, we initalize the agentscope config and load the corresponding model.

In [ ]:
from typing import Callable
import agentscope
agentscope.init(
    model_configs="./configs/model_configs.json",
    project="Conversation with NL2SQL",
)
from agentscope.manager import ModelManager
model_manager = ModelManager.get_instance()
loaded_model = model_manager.get_model_by_config_name('gpt-4')

Then, we init and provide the corresponding database path in sqlite file format.
Here we generate the sqlite file using the provided sql commands [here](./database/concert_singer/schema.sql). 
You can also use the .sqlite format file directly.

In [ ]:
from sql_utils import create_sqlite_db_from_schema
example_db_id = "concert_singer"
db_schema_path = "./database/concert_singer/schema.sql"
db_sqlite_path = "./database/concert_singer/concert_singer.sqlite"
create_sqlite_db_from_schema(db_schema_path, db_sqlite_path)


We can show a general description of the database schema using the code below.

In [3]:
from loguru import logger
from sql_utils import SQLPrompt
describe_prompt = SQLPrompt().describe_schema(db_sqlite_path)
sql_description = loaded_model([{"role": "assistant", "content": describe_prompt}]).text
logger.chat(sql_description)

The database consists of four tables: 

1. Stadium: This table includes the following fields: Stadium_ID (which is the primary key), Location, Name, Capacity, Highest, Lowest, and Average.

2. Singer: This table includes the following fields: Singer_ID (which is the primary key), Name, Country, Song_Name, Song_release_year, Age, and Is_male (a Boolean field that indicates the singer's gender).

3. Concert: This table is linked to the Stadium table via the Stadium_ID field. The table includes the fields concert_ID (which is the primary key), concert_Name, Theme, Stadium_ID, and Year.

4. Singer_in_Concert: This table acts as a junction table to establish a many-to-many relationship between the Singer and Concert tables. It includes the fields concert_ID and Singer_ID, which both make up the primary key for the table. These are also reference as foreign keys to their respective basetable: the Concert table via the concert_ID field, and the Singer table via the Singer_ID field.

In summar

## Step 2: Setting up tool functions
Then, we define the tools needed for React Agent to perform SQL query. Namely, our agent should be able to generate sql query given natural language and perform sql query to get the result.

- We referenced a third-party Text-to-SQL [DAIL-SQL](https://github.com/BeachWang/DAIL-SQL) to generate Text-to-SQL prompt.
- We use the query_sqlite service function in the agentscope.service module. 

In [4]:
from agentscope.service import (
    ServiceResponse, 
    ServiceExecStatus, 
    ServiceToolkit, 
    query_sqlite,
)
from sql_utils import DailSQLPromptGenerator


def generate_sql_query(question: str, db_path: str, model: Callable) -> ServiceResponse:
    """
    Generate the sql query based on the natural language question from user.
    
    Args:
        question (`str`):
            The question asked by user in natural language.
        db_path (`str`):
            The path to the .sqlite file.
        model (`Callable`):
            The LLM model used to generate the SQL query.
    """
    prompt_helper = DailSQLPromptGenerator(db_path)
    prepared_prompt = prompt_helper.generate_prompt({"content": question})
    
    def get_response_from_prompt(prompt: dict, model: Callable) -> str:
        """
        Generate response from prompt using LLM
        """
        messages = [{"role": "assistant", "content": prompt}]
        sql = model(messages).text
        sql = " ".join(sql.replace("\n", " ").split())
        sql = sql.strip().split("/*")[0]
        if sql.startswith("SELECT"):
            response = sql + "\n"
        elif sql.startswith(" "):
            response = "SELECT" + sql + "\n"
        else:
            response = "SELECT " + sql + "\n"
        return response

    sql_response = get_response_from_prompt(
        prepared_prompt["prompt"], model=model
    )

    return ServiceResponse(
        ServiceExecStatus.SUCCESS,
        sql_response
    )


# Use Service Toolkit to set up tool functions for LLMs
service_toolkit = ServiceToolkit()
service_toolkit.add(generate_sql_query, db_path=db_sqlite_path, model=loaded_model)
service_toolkit.add(query_sqlite, database=db_sqlite_path)

## Step 3: Create the ReAct agent to help perform SQL queries

Now, we can init the ReAct Agent using the defined tools and interact with the agent.

In [ ]:
from agentscope.agents import ReActAgent
agent = ReActAgent(
    name="assistant",
    model_config_name='gpt-4',
    service_toolkit=service_toolkit,
    sys_prompt="You are a helpful agent that preform SQL queries base on natual language instructions.",
    verbose=True, # set verbose to True to show the reasoning process
)

## Step 4: Interact with the SQL ReAct Agent using natural lanugage

Now let's see how the agent have response to our questions:

In [6]:
from agentscope.message import Msg

mss = Msg(
    name="user", 
    content="How many singers do we have?", 
    role="user"
)
logger.chat(mss)

sql_query_mss1 = agent(mss)

user: How many singers do we have?
##################### ITER 1, STEP 1: REASONING ######################
assistant: {
    "thought": "To answer the user's question, I need to find out how many unique singers are listed in the database. I will use the generate_sql_query function to create a SQL query to count distinct singers.",
    "speak": "Let me find this information for you.",
    "function": [
        {
            "name": "generate_sql_query",
            "arguments": {
                "question": "How many unique singers are we have in the database?"
            }
        }
    ]
}
####################### ITER 1, STEP 2: ACTION #######################
>>> Executing function generate_sql_query ...
>>> END 
system: Execution Results:
1. generate_sql_query:
    [EXECUTE STATUS]: SUCCESS
    [EXECUTE RESULT]: SELECT To find out how many unique singers are in the database, you would need to count the distinct singer IDs in the "singer" table. The appropriate SQL query for this would

In [7]:
mss = Msg(
    name="user", 
    content="Show all countries and the number of singers in each country.", 
    role="user"
)
logger.chat(mss)

sql_query_mss2 = agent(mss)

user: Show all countries and the number of singers in each country.
##################### ITER 1, STEP 1: REASONING ######################
assistant: {
    "thought": "To show all countries and the number of singers in each, I'll need to create and run an SQL query that counts singers by their country. This involves grouping the data by country and then counting the number of singers in each group.",
    "speak": "Let me find out how many singers there are in each country for you.",
    "function": [
        {
            "name": "generate_sql_query",
            "arguments": {
                "question": "Show all countries and the number of singers in each country."
            }
        }
    ]
}
####################### ITER 1, STEP 2: ACTION #######################
>>> Executing function generate_sql_query ...
>>> END 
system: Execution Results:
1. generate_sql_query:
    [EXECUTE STATUS]: SUCCESS
    [EXECUTE RESULT]: SELECT To show all countries and the number of singers in each 

In [8]:
mss = Msg(
    name="user", 
    content="How many singers have held concert in multiple stadiums? Give me the name of these singers.", 
    role="user"
)
logger.chat(mss)
sql_query_mss3 = agent(mss)

user: How many singers have held concert in multiple stadiums? Give me the name of these singers.
##################### ITER 1, STEP 1: REASONING ######################
assistant: {
    "thought": "To find out how many singers have held concerts in multiple stadiums and get their names, I need to create a SQL query that joins relevant tables to count the number of distinct stadiums for each singer. Singers with counts greater than one have held concerts in multiple stadiums. I will start by generating the SQL query.",
    "speak": "Let me find out which singers have held concerts in multiple stadiums for you.",
    "function": [
        {
            "name": "generate_sql_query",
            "arguments": {
                "question": "How many singers have held concerts in multiple stadiums and what are their names?"
            }
        }
    ]
}
####################### ITER 1, STEP 2: ACTION #######################
>>> Executing function generate_sql_query ...
>>> END 
system: Execu

You can use the following code to build a conversation with the ReAct agent:

``` python
from agentscope.agents import UserAgent

user = UserAgent(name="User")

x = None
while True:
    x = user(x)
    if x.content == "exit":
        break
    x = agent(x)
```

The capabilities of the ReAct agent are indeed very powerful. By modifying prompts and expanding the corresponding usage tools, the ReAct agent can leverage the combination of LLM and tools to accomplish more complex tasks.

We encourage users to delve deeper into exploring the Agent's functionalities and to try out the capabilities of the LLM agent on their own.